<img src="UNFC_COVER.jpg" alt="UNFC">

# Module 5: Linear Integer Programming

## Importing Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random

from scipy.optimize import linprog
from ortools.linear_solver import pywraplp
from pulp import *

## Mixed Integer Programming (MIP)

### Exercise 1: 

Google's OR-Tools which implements Branch and Bound internally for MIP problems!

Problem: Delivery Truck Loading Optimization

- Scenario: A logistics company has a delivery truck with limited space (capacity = $100$ units). They need to decide:

  - How many small boxes and large boxes to load.
  - Whether or not to rent an extra trailer (at a fixed rental cost).

- Product Details:
  - Small box:  
    - Volume = $5$ units
    - Profit = $\$10$ per box
  - Large box:
    - Volume = $20$ units
    - Profit = $\$30$ per box
  - Optional Trailer:
    - Increases capacity by $50$ units
    - Rental cost = $\$100$

#### Answer

Formulation:

- Decision Variables:
  - $ x_{1} $: number of small boxes loaded (integer $\ge 0$)
  - $ x_{2} $: number of large boxes loaded (integer $\ge 0$)
  - $ y $: whether trailer is rented (binary $0$ or $1$)

- Objective: Maximize Net Profit
$$
\text{Profit} = 10x_{1} + 30x_{2} - 100y
$$

- Constraints:

$$
5x_{1} + 20x_{2} \leq 100 + 50y \text{~ (Capacity)}
$$

$$
x_{1}, x_{2} \geq 0, \quad y \in \{0,1\} \text{~ (Non-negativity)}
$$

In [2]:
#small_box_volume = 5
#large_box_volume = 20
#small_box_profit = 10
#large_box_profit = 30
#capacity = 100
#optional_trailer_capacity = 50
#optional_trailer_cost = 100

In [1]:
from ortools.linear_solver import pywraplp

def delivery_truck_mip(
    small_box_volume = 5,
    large_box_volume = 20,
    small_box_profit = 10,
    large_box_profit = 30,
    capacity = 100,
    optional_trailer_capacity = 50,
    optional_trailer_cost = 100,
    ):
    # Create solver instance (CBC uses Branch & Bound internally)
    solver = pywraplp.Solver.CreateSolver('CBC')
    if not solver:
        return

    # Decision variables
    x1 = solver.IntVar(0, solver.infinity(), 'SmallBoxes')
    x2 = solver.IntVar(0, solver.infinity(), 'LargeBoxes')
    y = solver.IntVar(0, 1, 'TrailerRented')  # binary: 0 or 1

    # Capacity constraint
    #solver.Add(5 * x1 + 20 * x2 <= 100 + 50 * y)
    solver.Add(small_box_volume * x1 + large_box_volume * x2 <= capacity + optional_trailer_capacity * y)

    # Objective: Maximize net profit
    #solver.Maximize(10 * x1 + 30 * x2 - 100 * y)
    solver.Maximize(small_box_profit * x1 + large_box_profit * x2 - optional_trailer_cost * y)

    # Solve
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print(" Solution Found:")
        print(f"  Small boxes loaded: {x1.solution_value()}")
        print(f"  Large boxes loaded: {x2.solution_value()}")
        print(f"  Trailer rented (1=yes, 0=no): {y.solution_value()}")
        print(f"  Maximum Profit: ${solver.Objective().Value():.2f}")
    else:
        print(" No optimal solution found.")

In [2]:
# Run the optimization
#delivery_truck_mip()
delivery_truck_mip(
    small_box_volume = 5,
    large_box_volume = 20,
    small_box_profit = 10,
    large_box_profit = 30,
    capacity = 100,
    optional_trailer_capacity = 50,
    optional_trailer_cost = 100,
    )

 Solution Found:
  Small boxes loaded: 20.0
  Large boxes loaded: 0.0
  Trailer rented (1=yes, 0=no): 0.0
  Maximum Profit: $200.00


### Exercise 2: 

Problem: Warehouse Shipment Optimization

- Scenario: A logistics company manages shipments from **2 warehouses** to **3 retail stores**.
Each warehouse has a limited stock of goods, and each store has a specific demand.

  - Determine how many integer units to ship from each warehouse to each store to minimize shipping cost, while satisfying store demand and not exceeding warehouse capacity.

- Product Details:

| Warehouse | Supply (units) |
| --------- | -------------- |
| W1        | 50             |
| W2        | 60             |



| Store | Demand (units) |
| ----- | -------------- |
| S1    | 30             |
| S2    | 40             |
| S3    | 40             |



| Shipping Cost per Unit | S1 | S2 | S3 |
| ---------------------- | -- | -- | -- |
| W1                     | 4  | 6  | 9  |
| W2                     | 5  | 4  | 7  |

#### Answer 

- Mathematical Formulation

  - Decision Variables: 
    - Let $x_{ij}$: number of units shipped from warehouse $i$ to store $j$, for: $i \in \{1, 2\}$, $j \in \{1, 2, 3\}$. 
    - All $x_{ij}$ are non-negative integers

- Objective Function (Minimize cost):

$$
\text{Minimize } Z = 4x_{11} + 6x_{12} + 9x_{13} + 5x_{21} + 4x_{22} + 7x_{23}
$$

- Constraints:

$$
x_{11} + x_{12} + x_{13} \leq 50 \quad (\text{Supply W1}) 
$$

$$
x_{21} + x_{22} + x_{23} \leq 60 \quad (\text{Supply W2})
$$

$$
x_{11} + x_{21} = 30 \quad (\text{Demand S1})
$$

$$
x_{12} + x_{22} = 40 \quad (\text{Demand S2})
$$

$$
x_{13} + x_{23} = 40 \quad (\text{Demand S3})
$$

In [6]:
from ortools.linear_solver import pywraplp

def solve_shipping_problem():
    # Initialize solver
    solver = pywraplp.Solver.CreateSolver('SCIP')

    # Variables: x[i][j] = units from warehouse i to store j
    x = {}
    for i in range(2):
        for j in range(3):
            x[i, j] = solver.IntVar(0, solver.infinity(), f'x_{i+1}{j+1}')

    # Supply constraints
    solver.Add(x[0, 0] + x[0, 1] + x[0, 2] <= 50)  # W1
    solver.Add(x[1, 0] + x[1, 1] + x[1, 2] <= 60)  # W2

    # Demand constraints
    solver.Add(x[0, 0] + x[1, 0] == 30)  # S1
    solver.Add(x[0, 1] + x[1, 1] == 40)  # S2
    solver.Add(x[0, 2] + x[1, 2] == 40)  # S3

    # Cost matrix
    costs = [
        [4, 6, 9],  # W1 to S1, S2, S3
        [5, 4, 7]   # W2 to S1, S2, S3
    ]

    # Objective function
    objective = solver.Objective()
    for i in range(2):
        for j in range(3):
            objective.SetCoefficient(x[i, j], costs[i][j])
    objective.SetMinimization()

    # Solve
    status = solver.Solve()

    # Output
    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found:')
        total_cost = 0
        for i in range(2):
            for j in range(3):
                val = x[i, j].solution_value()
                print(f'\t Warehouse {i+1} → Store {j+1}: {int(val)} units')
                total_cost += val * costs[i][j]
        print(f'\t Total Shipping Cost = ${total_cost:.2f}')
    else:
        print('No optimal solution found.')

# Run the solver
solve_shipping_problem()

Optimal Solution Found:
	 Warehouse 1 → Store 1: 30 units
	 Warehouse 1 → Store 2: 0 units
	 Warehouse 1 → Store 3: 20 units
	 Warehouse 2 → Store 1: 0 units
	 Warehouse 2 → Store 2: 40 units
	 Warehouse 2 → Store 3: 20 units
	 Total Shipping Cost = $600.00
